In [1]:
# importing all the modules 

import sys
import math
import laspy
import pyproj

import numpy as np
import pandas as pd
import geopandas as gpd

from datetime import datetime
from pycrown import PyCrown
from shapely.geometry import Polygon

# Import my own functions
from my_functions.wcc import *
from my_functions.jucker import *

In [2]:
# Define the input folder where the files are stored
input_folder = 'input'

CHM_filename = 'clipped_CHM.tif'
DTM_filename = 'clipped_DTM.tif'
DSM_filename = 'clipped_DSM.tif'
LAS_filename = 'clipped_poincloud.las'
planting_types_filename = 'Planting Types.shp'
planting_years_filename = 'Shocott Spring Area.shp'


# Paths to the input files including DSM, DTM, CHM, and LiDAR point cloud,
# already clipped to the study site extent.
F_CHM = f'{input_folder}/{CHM_filename}'
F_DTM = f'{input_folder}/{DTM_filename}'
F_DSM = f'{input_folder}/{DSM_filename}'
F_LAS = f'{input_folder}/{LAS_filename}'

In [3]:
# Initialize the PyCrown class with the provided input files
# F_CHM: Path to the Canopy Height Model (CHM) file in .tif format
# F_DTM: Path to the Digital Terrain Model (DTM) file in .tif format
# F_DSM: Path to the Digital Surface Model (DSM) file in .tif format
# F_LAS: Path to the LiDAR point cloud file in .las format
# outpath: The directory where the output results will be saved
PC = PyCrown(F_CHM, F_DTM, F_DSM, F_LAS, outpath='result')

In [4]:
# Smoothing of CHM while preserving fine details in the CHM
# The method filter_chm applies a smoothing filter to the Canopy Height Model (CHM).
# Parameters:
# 1: The window size in pixels for smoothing. A value of 1 means no smoothing is applied,
#    as each pixel is considered individually without averaging neighboring values.

# ws_in_pixels=True: Specifies that the window size is given in pixels, not in some other unit.
# circular=False: Indicates that a square window is used for filtering instead of a circular one.

PC.filter_chm(1, ws_in_pixels=True, circular=False)

In [5]:
# Tree detection in the CHM using specified parameters
# The method tree_detection identifies tree tops in the Canopy Height Model (CHM).
# Parameters:
# PC.chm: The Canopy Height Model to be analyzed.
# ws=1: A window size of 1x1 pixel, meaning each pixel is considered independently as a potential tree top.
# hmin=1.3: The minimum height threshold for a pixel to be considered as a tree top. 
#   This has been adjusted from 16.0 to 1.3 meters to detect smaller trees.

PC.tree_detection(PC.chm, ws=1, hmin=1.3)

In [6]:
# Delineation of tree crowns using the specified algorithm and thresholds
# The method crown_delineation is used to define the boundaries of tree crowns in the CHM.

# Define parameters for crown delineation
algorithm = 'dalponteCIRC_numba'  # Algorithm used for crown delineation
th_tree = 1.3  # Minimum height threshold for considering a tree crown in meters
th_seed = 0.5  # Seed threshold factor
th_crown = 0.05  # Crown threshold factor
max_crown = 4.0  # Maximum allowable crown radius in meters

# Delineation of tree crowns using the specified algorithm and thresholds
PC.crown_delineation(algorithm=algorithm,
                     th_tree=th_tree,
                     th_seed=th_seed,
                     th_crown=th_crown,
                     max_crown=max_crown)

Tree crowns delineation: 0.003s


In [7]:
# Correcting tree tops after initial detection
# The correct_tree_tops() function refines the positions of detected tree tops.
# This process includes:
# - Removing false positives: tree tops that were incorrectly identified due to noise or low thresholds.
# - Adjusting the position of tree tops to more accurate locations within the tree crowns.
# - Merging closely positioned tree tops that likely represent the same tree.

# This correction step ensures that the final tree top positions are more accurate, 
# leading to better results in subsequent analyses.
PC.correct_tree_tops()

Number of trees: 6021
Tree tops corrected: 2551
Tree tops corrected: 42.36837734595582%
DSM correction: 475
COM correction: 2076


(475, 2076)

In [8]:
# Calculate tree height and elevation for the detected tree tops
PC.get_tree_height_elevation(loc='top')

# Calculate tree height and elevation for the corrected tree tops
PC.get_tree_height_elevation(loc='top_cor')

In [9]:
# Screen out small trees from the detected tree tops
# hmin=0.2: Minimum height threshold for a tree is 0.2 meters.
PC.screen_small_trees(hmin=0.2, loc='top') 

In [10]:
# Convert detected tree crowns to polygon shapes using a raster-based approach
PC.crowns_to_polys_raster()

# Convert detected tree crowns to smoothed polygon shapes
# and optionally store the corresponding LiDAR points for each crown
PC.crowns_to_polys_smooth(store_las=True)

Converting LAS point cloud to shapely points
Converting raster crowns to shapely polygons
Attach LiDAR points to corresponding crowns
Create convex hull around first return points
Classifying point cloud


In [11]:
# Perform quality control on the detected and processed tree crowns
PC.quality_control()

In [12]:
print(f"Number of trees detected: {len(PC.trees)}")

Number of trees detected: 6021


In [13]:
export_data = False  # or True, depending on your condition

if export_data:
    # Export the Canopy Height Model (CHM) as a raster file
    PC.export_raster(PC.chm, PC.outpath / 'chm.tif', 'CHM')

    # Export the locations of the initially detected tree tops
    PC.export_tree_locations(loc='top')

    # Export the locations of the corrected tree tops
    PC.export_tree_locations(loc='top_cor')

    # Export the tree crowns as raster-based polygons
    PC.export_tree_crowns(crowntype='crown_poly_raster')

    # Export the tree crowns as smoothed polygons
    PC.export_tree_crowns(crowntype='crown_poly_smooth')
else:
    print("Export skipped")

Export skipped


In [14]:
# AIM: Create a dataset with the identified trees, their heights and location (latitude and longitude)

# Convert your DataFrame to a GeoDataFrame using the 'top_cor' column for geometry
# 'top_cor' is assumed to be a column containing geometries (e.g., points representing tree tops)
trees_gdf = gpd.GeoDataFrame(PC.trees, geometry='top_cor')

# Setting the Coordinate Reference System (CRS) for the GeoDataFrame
# EPSG:27700 corresponds to OSGB36 / British National Grid, which is commonly used in the UK
epsg_code = "EPSG:27700"
trees_gdf = trees_gdf.set_crs(epsg_code)

# Function to calculate the diameter of a tree crown polygon
def polygon_diameter(polygon):
    if isinstance(polygon, Polygon):
        # Extract the bounding box of the polygon (minx, miny, maxx, maxy)
        minx, miny, maxx, maxy = polygon.bounds
        # Calculate the diameter as the maximum of the width or height of the bounding box
        diameter = max(maxx - minx, maxy - miny)
        return diameter
    else:
        return None

# Calculate the diameter for each tree crown polygon
# 'crown_poly_raster' is assumed to be a column containing polygon geometries for tree crowns
trees_gdf['diameter'] = trees_gdf['crown_poly_raster'].apply(polygon_diameter)


# Create a DataFrame with tree number, height, and calculated diameter
trees_gdf['tree_number'] = trees_gdf.index  # Assign a unique tree number based on the index
tree_database = trees_gdf[['tree_number', 'top_height', 'diameter']].copy()

# Add the geometry column back to the new DataFrame to make it a GeoDataFrame
tree_database = gpd.GeoDataFrame(tree_database, geometry=trees_gdf.geometry)

In [15]:
# AIM: add a shapefile to identify trees within particular areas and then planting years#
# This adds to the DataFrame the type and the year to each tree

# Loading the shapefile of the planting areas
planting_types = f'{input_folder}/{planting_types_filename}'
planting_areas_gdf = gpd.read_file(planting_types)

# Perform a spatial join to keep only trees within the planting areas
# 'how="inner"' keeps only the rows where the spatial join condition ('within') is true
trees_within_planting_areas = gpd.sjoin(tree_database, planting_areas_gdf, how='inner', op='within')

# Remove the 'index_right' and 'id' columns if present
# This cleans up the GeoDataFrame by removing unnecessary columns resulting from the join
columns_to_drop = ['index_right', 'id']
trees_within_planting_areas = trees_within_planting_areas.drop(columns=[col for col in columns_to_drop if col in trees_within_planting_areas.columns])

# planting years areas 
planting_years = f'{input_folder}/{planting_years_filename}'
planting_years_gdf = gpd.read_file(planting_years)

# Perform a spatial join to add attributes from the planting years shapefile
# Again, 'how="inner"' ensures that only trees within the specified areas are kept
trees_within_both_areas = gpd.sjoin(trees_within_planting_areas, planting_years_gdf, how='inner', op='within')

# Clean up by removing any redundant columns from the second spatial join
columns_to_drop_second = ['index_right']  # Adjust based on actual column names resulting from the second join
trees_within_both_areas = trees_within_both_areas.drop(columns=[col for col in columns_to_drop_second if col in trees_within_both_areas.columns])

# Rename columns to provide meaningful names
trees_within_both_areas = trees_within_both_areas.rename(columns={'id': 'Year'})

In [16]:
#len(trees_within_both_areas)
print(trees_within_both_areas.head(10))

    tree_number  top_height  diameter                       geometry  \
12           12    1.424002       1.0  POINT (507531.000 245983.000)   
16           16    1.940001       1.0  POINT (507489.000 245960.000)   
18           18    2.199003       1.0  POINT (507581.000 245946.000)   
19           19    1.453003       1.0  POINT (507560.000 245938.000)   
20           20    2.310001       1.0  POINT (507611.000 245938.000)   
21           21    1.770002       1.0  POINT (507499.000 245930.000)   
22           22    1.814003       1.0  POINT (507480.000 245914.000)   
23           23    1.422001       1.0  POINT (507492.000 245911.000)   
24           24    1.467001       1.0  POINT (507543.000 245908.000)   
25           25    1.815001       1.0  POINT (507496.000 245890.000)   

          Type  Year  
12  Shrub-rich     3  
16  Shrub-rich     3  
18  Shrub-rich     3  
19  Shrub-rich     3  
20  Shrub-rich     3  
21  Shrub-rich     3  
22  Shrub-rich     3  
23  Shrub-rich     3  


In [17]:
# Create a CSV file for further analysis
trees_within_both_areas.to_csv('trees_within_both_areas.csv', index=False)

In [18]:
# Reading the CSV file into a DataFrame
df = trees_within_both_areas

In [19]:
# AIM: add DBH to the data set of each tree
# This cell is based on Jucker et al. 2017 ("Allometric equations for integrating remote sensing imagery into forest monitoring programmes")

# Parameters for DBH calculation
exp_factor_d = np.exp(0.056**2 / 2)

# Calculate DBH (Diameter at Breast Height)
# Unit for DBH is cm 
df['DBH'] = 0.557 * (df['top_height'] * df['diameter'])**0.809 * exp_factor_d

# WCC FRAMEWORK #

In [20]:
# Initialize total aggregates
total_agb_all_species = 0
total_root_biomass_all_species = 0

# Define species percentages and planting mix
species_percentages = {
    'oak': 0.40,  # 40% Oak
    'Scots pine': 0.30,  # 30% Scots Pine
    'European larch': 0.30  # 30% European Larch
}
planting_mix = 'Mixed Wood'
year = 2



# Assuming `df` is your DataFrame containing the tree data
tree_stats = tree_statistics(df, species_percentages, planting_mix, year)

print_tree_statistics(tree_stats)

# Iterate over all species in your dataset
for species in tree_stats['species_distribution'].keys():
    # Get the volume statistics for the species
    volume_stats = calculate_tariff_numbers_and_volume(tree_stats, species)

    # Calculate the biomass for the species
    biomass_stats = calculate_biomass(tree_stats, species, volume_stats)

    # Accumulate the total AGB and root biomass for all species
    total_agb_all_species += biomass_stats['total_AGB']
    total_root_biomass_all_species += biomass_stats['total_root_biomass']

# Compute the ratio of total root biomass to total AGB
root_to_agb_ratio = total_root_biomass_all_species / total_agb_all_species if total_agb_all_species != 0 else None

# Print the results
print(f"Total AGB for all species: {total_agb_all_species:.4f} oven-dry tonnes")
print(f"Total Root Biomass for all species: {total_root_biomass_all_species:.4f} oven-dry tonnes")
if root_to_agb_ratio is not None:
    print(f"Ratio of Total Root Biomass to Total AGB: {root_to_agb_ratio:.4f}")
else:
    print("Total AGB is zero, cannot compute ratio.")

Tree Statistics:
Total number of trees: 4
Total number of saplings: 2145
Total number of large trees: 0
Mean Tree Height for Trees: 5.86 meters
Mean DBH for Trees: 7.48 cm
Mean Tree Height for Saplings: 3.15 meters
Mean DBH for Saplings: 2.46 cm
No large trees identified.
Quadratic Mean DBH for Trees: 7.5 cm
Mean Basal Area for Trees: 0.0044 m^2

Total AGB for all species: 0.0300 oven-dry tonnes
Total Root Biomass for all species: 0.0050 oven-dry tonnes
Ratio of Total Root Biomass to Total AGB: 0.1667


In [22]:
# Calculate tree biomass and convert to carbon and CO2 for each species
total_aggregates = calculate_and_print_species_biomass_and_carbon(tree_stats, species_percentages)
# Print overall totals

print_overall_totals(
    total_aggregates['total_biomass_all_species'], 
    total_aggregates['total_carbon_all_species'], 
    total_aggregates['total_co2_all_species'], 
    total_aggregates['total_co2_trees_all_species'], 
    total_aggregates['total_co2_saplings_all_species']
)

Species: Oak
Number of Trees: 2
Number of Saplings: 859
Tariff Number: 17
Mean Tree Volume: 0.0080 m^3
Total Stem Volume: 0.0209 m^3
Total Stem Biomass: 0.0117 oven-dry tonnes
Total Crown Biomass: 0.0049 oven-dry tonnes
Total Root Biomass: 0.0000 oven-dry tonnes
Total Above-Ground Biomass (AGB): 0.0166 oven-dry tonnes
Total Biomass: 0.0166 oven-dry tonnes
Total Carbon Content: 0.0083 tonnes C
Total CO2 Content for Trees: 0.0304 tonnes CO2
Total CO2 Content for Saplings: 0.0000 tonnes CO2
-------------------------------------------------- 

Species: Scots pine
Number of Trees: 1
Number of Saplings: 643
Tariff Number: 15
Mean Tree Volume: 0.0077 m^3
Total Stem Volume: 0.0100 m^3
Total Stem Biomass: 0.0042 oven-dry tonnes
Total Crown Biomass: 0.0024 oven-dry tonnes
Total Root Biomass: 0.0024 oven-dry tonnes
Total Above-Ground Biomass (AGB): 0.0065 oven-dry tonnes
Total Biomass: 0.0089 oven-dry tonnes
Total Carbon Content: 0.4000 tonnes C
Total CO2 Content for Trees: 0.0163 tonnes CO2
Tota

#  JUCKER ET AL. 2017 

### AGB- using crown diameter and height

In [23]:
# Parameters for ABG 
exp_factor_agb = np.exp(0.204**2 / 2)

# As there is no option to classify the type of species of signular tree, it will be assumed that they could be equally (50% chance) either angiosperms and gymnosperms and then a waighted average will be applied  
# Calculate AGB for angiosperms and gymnosperms
# Unit for AGB is kg.
df['AGB_angiosperm'] = 0.016 * (df['top_height'] * df['diameter'])**2.013 * exp_factor_agb
df['AGB_gymnosperm'] = 0.109 * (df['top_height'] * df['diameter'])**1.790 * exp_factor_agb


In [24]:
species_mix_proportions = {
    "Mixed Wood": (0.4, 0.6),  # 40% Angiosperms, 60% Gymnosperms
    "Native Bro": (1.0, 0.0),  # 100% Angiosperms
    "Native Shr": (1.0, 0.0),  # 100% Angiosperms
    "Native Tim": (1.0, 0.0),  # 100% Angiosperms
    "Shrub-rich": (1.0, 0.0)   # 100% Angiosperms
}

ratio_bgb_to_agb = root_to_agb_ratio  # Given ratio of BGB to AGB

# Calculate the biomass summary
biomass_summary = calculate_biomass_summary(df, ratio_bgb_to_agb, species_mix_proportions)

In [25]:
# Display the summary
print("Biomass and Carbon Content Summary by Planting Type and Year:")
print(biomass_summary)

Biomass and Carbon Content Summary by Planting Type and Year:
          Type  Year  Total_AGB  Mean_AGB  Number_of_Trees  Total_BGB  \
0   Mixed Wood     1  29.933875  0.011408             2624   4.990948   
1   Mixed Wood     2   5.848508  0.002722             2149   0.975136   
2   Native Bro     1   0.173455  0.000899              193   0.028920   
3   Native Bro     2   0.025474  0.000349               73   0.004247   
4   Native Bro     3   0.000115  0.000038                3   0.000019   
5   Native Shr     1   0.002540  0.000254               10   0.000423   
6   Native Shr     2   0.005806  0.001935                3   0.000968   
7   Native Shr     3   0.000378  0.000189                2   0.000063   
8   Native Tim     1   0.041157  0.000490               84   0.006862   
9   Native Tim     2   0.027150  0.000272              100   0.004527   
10  Shrub-rich     1   0.023496  0.000691               34   0.003918   
11  Shrub-rich     2   0.000158  0.000079                2   0